In [1]:
import pandas as pd
import numpy as np
from os import path
import import_ipynb
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
import torch
from matplotlib import pyplot as plt
from pandas.tseries.offsets import BusinessDay as BDay
from pandas.tseries.offsets import Day 
import ast

In [2]:
import utils
import models
# import l2lutils
import optutils

importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from models.ipynb
importing Jupyter notebook from l2lutils.ipynb
importing Jupyter notebook from optutils.ipynb


In [3]:
#import compute_features
#import feature_utils as fu
#from paths import LEARN_DATA,INDEX_FILE,REGR_DATA,INFER_DATA, INFER_DATA_PRE
#from compute_features import compute_loss,add_trade_features,feature_from_featureList
#from compute_features import drop_features_regex_list,aug_from_orig,rem_orig_keep_oi_pcr
#from compute_features import self_history,rem_zero_contracts
#from compute_features import TxSeq

In [4]:
def options_data(optype='CE',yrL=['2016','2017','2018','2019','2020'],qtrL=['Q1','Q2','Q3','Q4'],minDate=None,maxDate=None,
                features=[1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0],task='classification',
                data_location=None,suffix=None,appendTo=None,orgdate=None,enddate=None):
    txseq = TxSeq()
    txseq.registerFlist([compute_loss,add_trade_features,feature_from_featureList,
                         drop_features_regex_list,aug_from_orig,rem_orig_keep_oi_pcr,
                         self_history,rem_zero_contracts])
    dfL=[]
    if appendTo != None: dfL+=[appendTo]
    for yr in yrL:
        for qtr in qtrL:
            #yr, qtr, optype = '2020','Q4','CE'
            yqtr = yr+qtr
            txseq.set(yr,qtr,optype)
            txseq.clear()
            txseq.load(txlist=features,filename='./transforms.csv')
            if data_location == None: filename = LEARN_DATA+yqtr+optype+'data.csv'
            else:
                filename = data_location+yqtr+optype+suffix+'.csv'
            for i,tx in enumerate(txseq.txL):
                if tx['func']=='aug_from_orig':
                    ad=ast.literal_eval(tx['adict'])
                    if suffix=='meta': 
                        ad['curloc']='meta/'
                        ad['orgdate']=orgdate
                        ad['enddate']=enddate
                    elif suffix=='data': 
                        ad['curloc']='daily/'
                        ad['curdate']=qtr
                    txseq.txL[i]['adict']=str(ad)
            if path.exists(filename):
                df = pd.read_csv(filename)
                if minDate != None: 
                    df = df.loc[pd.to_datetime(df['Date'])>=pd.to_datetime(minDate)]
                if maxDate != None:
                    df = df.loc[pd.to_datetime(df['Date'])<=pd.to_datetime(maxDate)]
                #print(df.shape)
                df = txseq.apply(df)
                dfL += [df]
    dfcat = pd.concat(dfL,axis=0)
    return dfcat

In [5]:
class OptionsDS():
    def __init__(self,df):
        self.df=df
        self.scaler=StandardScaler()
        self.startDate=min(pd.to_datetime(self.df['Date']))
        self.endDate=max(pd.to_datetime(self.df['Date']))
        self.meta_train_tasks=[]
        self.meta_test_tasks=[]
    def compute_tr_dates(self,date='01-Jan-2019',length=10):
        dt = pd.to_datetime(date)
        tr_dt = dt-Day(31)-BDay(length)
        return (tr_dt,tr_dt+BDay(30))
    def create_train_test(self,minDate='01-Oct-2016',maxTrainDate='31-Oct-2016',
                         minTestDate='01-Dec-2016',maxTestDate='31-Dec-2016'):
        df=self.df
        dftrain=df.loc[(pd.to_datetime(df['Date'])>=pd.to_datetime(minDate))&
                   (pd.to_datetime(df['Date'])<=pd.to_datetime(maxTrainDate))]
        dftest=df.loc[(pd.to_datetime(df['Date'])>=pd.to_datetime(minTestDate))&
                   (pd.to_datetime(df['Date'])<=pd.to_datetime(maxTestDate))]
        dfgap=df.loc[(pd.to_datetime(df['Date'])>pd.to_datetime(maxTrainDate))&
                   (pd.to_datetime(df['Date'])<pd.to_datetime(minTestDate))]
        return dftrain,dfgap,dftest
    def create_samples(self,dftrain,dfgap,dftest):
        if 'Breach' in dftrain.columns:
            dftrain=dftrain.drop(columns=['Breach','NBreach','Date','Expiry'])
            dfgap=dfgap.drop(columns=['Breach','NBreach','Date','Expiry'])
            dftest=dftest.drop(columns=['Breach','NBreach','Date','Expiry'])
        else:
            dftrain=dftrain.drop(columns=['Date','Expiry'])
            dfgap=dfgap.drop(columns=['Date','Expiry'])
            dftest=dftest.drop(columns=['Date','Expiry'])
        y_gap=np.ones(dfgap.shape[0])/2
        l_gap=np.zeros(dfgap.shape[0])
        if 'ExInRange' in dftrain.columns:
            y_train=1-dftrain['ExInRange'].values
            l_train=dftrain['Loss'].values
            y_test=1-dftest['ExInRange'].values
            l_test=dftest['Loss'].values
            dftrain=dftrain.drop(columns=['ExInRange','Loss'])
            dfgap=dfgap.drop(columns=['ExInRange','Loss'])
            dftest=dftest.drop(columns=['ExInRange','Loss'])
        else:
            y_train=y_gap
            l_train=l_gap
            y_test=y_gap
            l_test=l_gap
        X_train=dftrain.values
        X_gap=dfgap.values
        X_test=dftest.values
        self.scaler.fit(X_train)
        X_train=self.scaler.transform(X_train)
        X_gap=self.scaler.transform(X_gap)
        X_test=self.scaler.transform(X_test)
        return (X_train,y_train),(X_gap,y_gap),(X_test,y_test)
    def normal_train_test(self,minDate='01-Jan-2019',maxTrainDate='31-Jan-2019',
                         minTestDate='01-Mar-2019',maxTestDate='31-Mar-2019',
                         batch_size=32):
        dftrain,dfgap,dftest=self.create_train_test(minDate,maxTrainDate,
                                minTestDate,maxTestDate)
        (X_train,y_train),(X_gap,y_gap),(X_test,y_test)=self.create_samples(dftrain,
                                                                            dfgap,dftest)
        trainDS=utils.MyDS(X_train,y_train)
        testDS=utils.MyDS(X_test,y_test)
        train_loader=torch.utils.data.DataLoader(trainDS,batch_size=batch_size)
        return train_loader,trainDS,testDS
    def create_tasks(self,trainStart='01-Jan-2019',trainEnd='31-Jan-2019',
                       testStart='01-Feb-2019',testEnd='28-Feb-2019',stride=1,length=10,
                    ONLYTEST=False):
        dt=pd.to_datetime(trainStart)
        while (dt<=pd.to_datetime(trainEnd)):
            if ONLYTEST: break
            minDate=dt
            #print('Train Task',dt)
            maxTrainDate=minDate+BDay(length)
            minTestDate=maxTrainDate+Day(30)
            maxTestDate=minTestDate+BDay(length)
            dftrain,dfgap,dftest=self.create_train_test(minDate,maxTrainDate,
                                minTestDate,maxTestDate)
            task={'dftrain':dftrain,'dfgap':dfgap,'dftest':dftest}
            self.meta_train_tasks=self.meta_train_tasks+[task]
            dt=dt+BDay(stride)
        dt=pd.to_datetime(testStart)
        if ONLYTEST: 
            dt=self.compute_tr_dates(testStart)[0]
            testEnd=dt
        while (dt<=pd.to_datetime(testEnd)):
            minDate=dt
            #print('Test Task',dt)
            if not ONLYTEST:
                maxTrainDate=minDate+BDay(length)
                minTestDate=maxTrainDate+Day(30)
                maxTestDate=minTestDate+BDay(length)
            elif ONLYTEST:
                maxTrainDate=minDate+BDay(length)
                minTestDate=pd.to_datetime(testStart)
                maxTestDate=minTestDate
            dftrain,dfgap,dftest=self.create_train_test(minDate,maxTrainDate,
                                minTestDate,maxTestDate)
            task={'dftrain':dftrain,'dfgap':dfgap,'dftest':dftest}
            self.meta_test_tasks=self.meta_test_tasks+[task]
            dt=dt+BDay(stride)
        return
    def get_meta_train_task(self,r=None):
        n=len(self.meta_train_tasks)
        if n==0: return -1
        if r==None: r=np.random.randint(0,n)
        return self.meta_train_tasks[r]
    def get_meta_test_task(self,r=None):
        n=len(self.meta_test_tasks)
        if r==None: r=np.random.randint(0,n)
        return self.meta_test_tasks[r]

In [6]:
class OptionsTaskLoader():
    def __init__(self,opDS,getfn):
        self.opDS = opDS
        self.getfn = getfn
    def get_task(self,r=None):
        taskD=self.getfn(r=r)
        if taskD==-1:
            return -1
        dftrain=taskD['dftrain']
        dftest=taskD['dftest']
        dfgap=taskD['dfgap']
        (X_train,y_train),(X_gap,y_gap),(X_test,y_test)=self.opDS.create_samples(dftrain,
                                                                            dfgap,dftest)
        X_trainT = torch.Tensor(X_train)
        y_trainT = torch.LongTensor(y_train)
        X_gapT = torch.Tensor(X_gap)
        y_gapT = torch.LongTensor(y_gap)
        X_testT = torch.Tensor(X_test)
        y_testT = torch.LongTensor(y_test)
        self.scaler = self.opDS.scaler
        return (X_trainT,y_trainT),(X_testT,y_testT)